In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator

In [ ]:
def calculate_entropy(feature):
  """
  Calculate the entropy of a given feature

  parameters:
  -feature: list containing the value of feature.

  Returns:
  entropy_value: The entropy of the feature.
  """
  # Check if there is only one or zero labels in the feature.

  if len(feature)<=1:
    return 0

  # count the number of occurences of each class
  total_count=np.bincount(feature)
  #count the probabilities of each class label
  probabilities=total_count[np.nonzero(total_count)]/len(feature)
  if len(probabilities)<=1: # handling if the length of the probabilties is less than or equal 1
    return 0
    # computing the entropy using the entropy equation.
  return -np.sum(probabilities*np.log2(probabilities))/np.log2(len(probabilities)) #entropy equation


In [ ]:
def calculate_information_gain(previous_labels, current_labels):
    """
    Calculate the information gain between two sets of labels.

    Parameters:
    - previous_labels: List containing the labels before splitting.
    - current_labels: List of lists containing subsets of labels after splitting.

    Returns:
    - The information gain achieved by splitting the dataset based on the given subsets.
    """
    conditional_entropy = 0

    # Calculate the conditional entropy for each subset of labels in current_labels
    for labels_subset in current_labels:
        conditional_entropy += (calculate_entropy(labels_subset) * len(labels_subset) / len(previous_labels))

    # return the information gain by subtracting conditional entropy from previous entropy
    return calculate_entropy(previous_labels) - conditional_entropy


In [ ]:
def split_classes(X,y,split_attribute,split_val):
  """
  Split the dataset into two subsets based on a split value and split attribute.

  Parameters:
  - X: list of the features.
  - y: list of the target variables.
  - split_attribute: Index of the column used for splitting.
  - split_val: value used for splitting the data.

  Returns:
  - features_left: list of the left subset.
  - features_right: list of the right subset.
  - target_variables_left: list of target variables of the left subset
  - target_variables_right: list of the target variables of the right subset
  """
  X=np.array(X) # converting X to be an array
  # getting the column specified by the split_attribute
  column_split=X[:,split_attribute]
  # lists to store the splitted data
  features_left,features_right,target_variables_left,target_variables_right=[],[],[],[]
  index=0
  if isinstance(split_val,str)==False: # numerical
    for i in column_split:
      if i<=split_val:
        features_left.append(X[index])
        target_variables_left.append(y[index])
      else:
        features_right.append(X[index])
        target_variables_right.append(y[index])
      index+=1

  return features_left,features_right,target_variables_left,target_variables_right



In [ ]:
def find_best_split(X,y,split_attribute):
  """
  Find the best split value and information gain for a given attribute.

  Parameters:
  - X: list of features.
  - y: list of the target variables.
  - split_attribute: index of column used for splitting.

  Returns:
  - best_split_val: best split value found.
  - best_information_gain: best information gain found.
  """
  best_information_gain=0
  X=np.array(X)
  column_split=X[:,split_attribute] # extract the columns specified by the split_attribute
  column_split=np.unique(column_split) #  get the unique values only
  best_split_val=None
  for split_val in column_split:
    current_X_left,current_X_right,current_y_left,current_y_right=split_classes(X,y,split_attribute,split_val)
    current_y=[]
    current_y.append(current_y_left)
    current_y.append(current_y_right)

    current_info_gain=calculate_information_gain(y,current_y)
    # update the best split and information gain if current split is better
    if current_info_gain>best_information_gain:
      best_information_gain=current_info_gain
      best_split_val=split_val
  return best_split_val,best_information_gain



In [ ]:
def find_best_feature(X,y):
  """
  Find the best feature and split value for splitting the data.

  Parameters:
  - X: list of the features.
  - y: list of the target variables

  Returns:
  - best_feature: index of the best feature found.
  - best_split_val: best split value for the best feature.
  """
  best_info_gain=0
  best_feature= None
  best_split_val= None
  for feature_index in range(len(X[0])):
    current_best_split_val,current_best_information_gain=find_best_split(X,y,feature_index)
    if current_best_information_gain>best_info_gain:
      best_info_gain=current_best_information_gain
      best_feature=feature_index
      best_split_val=current_best_split_val
  return best_feature,best_split_val





# Decision Tree ID3 Class


In [ ]:
class MyDecisionTree(object):
  def __init__(self,max_depth=None):
    """
    Initialize the decision tree.

    Parameters:
    - max_depth: Maximum depth of the decision tree. If None, the tree will be grown until all leaves are pure.

    """
    self.tree={}
    self.residual_tree={}
    self.max_depth=max_depth


  def fit(self,X,y,depth):
    """
    Build the decision tree recursively.

    Parameters:
    - X: Features list.
    - y: target variables list.
    - depth: Current depth of the tree.

    Returns:
    - tree: The trained decision tree.

    """
    unique_labels=np.unique(y)
    if (len(unique_labels)==1)or (depth==self.max_depth):
      unique_labels,count_unique_labels=np.unique(y,return_counts=True)
      index=count_unique_labels.argmax()
      classification=unique_labels[index]
      return classification

    best_feat,best_split=find_best_feature(X,y)
    features_left,features_right,target_variables_left,target_variables_right=split_classes(X,y,best_feat,best_split)

    if not isinstance(best_split,str):
      question="{} == {}".format(best_feat,best_split)
    node={question: []}

    depth+=1
    yes_answer=self.fit(features_left,target_variables_left,depth)
    no_answer=self.fit(features_right,target_variables_right,depth)

    if yes_answer==no_answer:  # Both trees are the same
      node=yes_answer
    else:
      node[question].append(yes_answer)
      node[question].append(no_answer)
    self.tree=node
    return node


  def predict(self,record,flag=1):
    """
    Predict the label of a given record using the decision tree.

    Parameters:
    - record: The input record to predict its label.
    - flag: Flag to indicate if it's the first time calling the method.

    Returns:
    - prediction: The predicted label of the input record.
    """
    if flag==1: ## first time
      self.residual_tree=self.tree
    question=list(self.residual_tree.keys())[0]
    feature,comparison,value=question.split()

    if comparison=="==": # Integer
      if record[int(feature)]<=float(value):
        answer=self.residual_tree[question][0]
      else:
        answer=self.residual_tree[question][1]
    # base case
    if not isinstance(answer,dict):
      return answer
    else:
      self.residual_tree=answer
      return self.predict(record,0) # flag =0

  def print_tree(self, node=None, indent=""):
        """
        Recursively print the decision tree.

        Parameters:
        - node: Current node of the tree (default is the root node).
        - indent: Indentation for better visualization (default is empty string).
        """
        if node is None:
          node = self.tree

        if isinstance(node, dict):
            question = list(node.keys())[0]
            feature_index, comparison, value = question.split()
            feature_name = self.feature_names[int(feature_index)]
            print(indent + "Question:" + feature_name + " " + comparison + " " + value + "?")
            yes_answer = node[question][0]
            no_answer = node[question][1]
            print(indent + "--> Yes:")
            self.print_tree(yes_answer, indent + "  ")
            print(indent + "--> No:")
            self.print_tree(no_answer, indent + "  ")
        else:
            print(indent + "Predicted class:", node)



In [ ]:
def tree_accuracy(id3,X,y):
  y_predicted=[]
  for record in X:
    y_predicted.append(id3.predict(record))

  results=[prediction==truth for prediction,truth in zip(y_predicted,y)]
  accuracy=float(results.count(True))/float(len(results))
  print("accuracy: %.4f" % accuracy)
  return accuracy

# Section 1: Data reading and preprocessing

In [ ]:
df=pd.read_csv('cardio_train.csv')
df


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62,110,80,1,1,0,0,1,0
1,1,20228,1,156,85,140,90,3,1,0,0,1,1
2,2,18857,1,165,64,130,70,3,1,0,0,0,1
3,3,17623,2,169,82,150,100,1,1,0,0,1,1
4,4,17474,1,156,56,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72,135,80,1,2,0,0,0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   id           70000 non-null  int64
 1   age          70000 non-null  int64
 2   gender       70000 non-null  int64
 3   height       70000 non-null  int64
 4   weight       70000 non-null  int64
 5   ap_hi        70000 non-null  int64
 6   ap_lo        70000 non-null  int64
 7   cholesterol  70000 non-null  int64
 8   gluc         70000 non-null  int64
 9   smoke        70000 non-null  int64
 10  alco         70000 non-null  int64
 11  active       70000 non-null  int64
 12  cardio       70000 non-null  int64
dtypes: int64(13)
memory usage: 6.9 MB


In [ ]:
df.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,1.349571,164.359229,74.205543,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,0.476838,8.210126,14.395829,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.isna().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

(70000, 13)

In [ ]:
df['cardio'].value_counts()

cardio
0    35021
1    34979
Name: count, dtype: int64

# Section 2: Results on the kaggle dataset

In [ ]:
X=df.drop(columns=['cardio','id','ap_lo','age','height','weight','ap_hi'],axis=1)
y=df['cardio']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_test,y_train,y_test=X_train.to_numpy(),X_test.to_numpy(),y_train.to_numpy(),y_test.to_numpy()

max_depth=6
initial_depth=0
id3=MyDecisionTree(max_depth)
start_time = time.time()
id3.fit(X_train, y_train, initial_depth)
training_time_id3 = time.time() - start_time

start_time = time.time()
accuracy_id3 = tree_accuracy(id3, X_test, y_test)
prediction_time_id3 = time.time() - start_time

print("My Implementation from scratch:")
print("Accuracy:", accuracy_id3)
print("Training Time in sec:", training_time_id3)
print("Prediction Time:", prediction_time_id3)



accuracy: 0.5974
My Implementation from scratch:
Accuracy: 0.5973571428571428
Training Time in sec: 2.2202165126800537
Prediction Time: 0.11826014518737793


# Section 3: sklearn results on the kaggle dataset

In [ ]:
start_time=time.time()
tree_sklearn=Id3Estimator()
tree_sklearn.fit(X_train,y_train)
training_time_sklearn=time.time()-start_time

start_time=time.time()
prediction_sklearn=tree_sklearn.predict(X_test)
prediction_time_sklearn=time.time()-start_time

accuracy_sklearn = accuracy_score(y_test, prediction_sklearn)
print("Sklearn implementation:")
print("Accuracy:", accuracy_sklearn)
print("Training Time in sec:", training_time_sklearn)
print("Prediction Time:", prediction_time_sklearn)

Sklearn implementation:
Accuracy: 0.5925714285714285
Training Time in sec: 0.25057220458984375
Prediction Time: 0.02856612205505371


# Section 4: results of tree implemented from scratch on student data

In [ ]:
df2=pd.read_csv('ML assignment 6.csv')
df2


,Early registration,Finished homework,Senior,Likes Coffee,Liked The Last homework,A
0,1,1,0,0,1,1
1,1,1,1,0,1,1
2,0,0,1,0,0,0
3,0,1,1,0,1,0
4,0,1,1,0,0,1
5,0,0,1,1,1,1
6,1,0,0,0,1,0
7,0,1,0,1,1,1
8,0,0,1,0,1,1
9,1,0,0,0,0,0


In [ ]:
X=df2.drop(['A'],axis=1)
y=df2['A']

X=X.to_numpy()
y=y.to_numpy()
max_depth=6
initial_depth=0
id3=MyDecisionTree(max_depth)
id3.feature_names = ["Early registration", "Finished homework", "Senior","Likes Coffee","Liked The Last homework"]
id3.fit(X,y,initial_depth)



{'1 == 0': [{'3 == 0': [{'2 == 0': [0, {'4 == 0': [0, 1]}]}, 1]},
  {'0 == 0': [{'2 == 0': [1, {'4 == 0': [1, 0]}]}, 1]}]}

In [ ]:
# id3.tree
id3.print_tree()


Question:Finished homework == 0?
--> Yes:
  Question:Likes Coffee == 0?
  --> Yes:
    Question:Senior == 0?
    --> Yes:
      Predicted class: 0
    --> No:
      Question:Liked The Last homework == 0?
      --> Yes:
        Predicted class: 0
      --> No:
        Predicted class: 1
  --> No:
    Predicted class: 1
--> No:
  Question:Early registration == 0?
  --> Yes:
    Question:Senior == 0?
    --> Yes:
      Predicted class: 1
    --> No:
      Question:Liked The Last homework == 0?
      --> Yes:
        Predicted class: 1
      --> No:
        Predicted class: 0
  --> No:
    Predicted class: 1
